In [2]:
import xgboost as xgb
import pickle
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import wordnet
from collections import Counter
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
from math import sqrt

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pf416e\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
train = pd.read_csv('train_new.csv')
test = pd.read_csv('test_new.csv')

org = pd.read_excel('Final_Train.xlsx')
target=org.Fees

In [207]:
df_new = pd.concat([train, test])

##One hot encoding
dfmerged = pd.concat([df_new[['Experience1','Rating1', 'Feedback1']],
           pd.get_dummies(df_new['Profile'],drop_first = True),pd.get_dummies(df_new['Places'],drop_first = True),pd.get_dummies(df_new['City'],drop_first = True),pd.get_dummies(df_new['Miscellaneous_Info8'],drop_first = True),pd.get_dummies(df_new['Qualification8'],drop_first = True)],axis=1)

dfmerged.columns = np.arange(len(dfmerged.columns))

train1=dfmerged[:5961]
test1=dfmerged[5961:7948]

X_train, X_validation, y_train, y_validation = train_test_split(train1, target, train_size=0.7, random_state=1234)

In [208]:
model=xgb.XGBRegressor(colsample_bylevel=0.9, 
                       colsample_bytree=0.9,
                       gamma=1, 
                       learning_rate=0.1, 
                       max_depth=10,
                       min_child_weight=1, 
                       n_estimators=200, 
                       objective='reg:linear', 
                       reg_alpha=1, 
                       reg_lambda=1,
                       scale_pos_weight=1, 
                       seed=99, 
                       subsample=0.85,
                       silent=True)
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample_bytree=0.9, gamma=1, learning_rate=0.1, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=1, reg_lambda=1, scale_pos_weight=1, seed=99, silent=True,
       subsample=0.85)

In [209]:
print(model.score(X_train, y_train))
print(model.score(X_validation,y_validation))

0.6767294796091401
0.2626712265799812


In [210]:
# save model to file
pickle.dump(model, open("basic_ohc.pickle.dat", "wb"))

In [211]:
#Model Loading
xgb_ld = pickle.load(open("basic_ohc.pickle.dat", "rb"))

In [212]:
ypred = xgb_ld.predict(X_train)
ypred_val = xgb_ld.predict(X_validation)


print(sqrt(mean_squared_error(y_train, ypred)))
print(sqrt(mean_squared_error(y_validation, ypred_val)))

108.2456155252583
164.9629616980957


<h>**Data For Stacking**</h>

In [213]:
#Loading the model
xgb_ld = pickle.load(open("basic_ohc.pickle.dat", "rb"))

y_all=xgb_ld.predict(train1)
y_all=pd.DataFrame(y_all)

y_all.columns=['xgb_wocat']

y_all.to_csv('xgb_wocat_stck_train.csv', index=False)

In [214]:
#Without Adjustment
y_pred_test2=xgb_ld.predict(test1)
y_pred_test2=pd.DataFrame(y_pred_test2)
y_pred_test2.columns=['Fee_pred']

<h>**Validation Data For Stacking**</h>

In [215]:
xgb_ld = pickle.load(open("basic_ohc.pickle.dat", "rb"))

y_predicted_test=y_all=xgb_ld.predict(test1)
y_predicted_test=pd.DataFrame(y_predicted_test)
y_predicted_test.columns=['xgb_wocat']
y_predicted_test.to_csv('xgb_wocat_stck_val.csv', index=False)

In [216]:
#With Adjustment
fee_test = pd.read_excel('Final_Test.xlsx')
fee_test=fee_test['Miscellaneous_Info']
fee_test1=pd.DataFrame(fee_test)

for i in range(1987):
    if (pd.isnull(fee_test1['Miscellaneous_Info'][i])==False):
        myString=fee_test1['Miscellaneous_Info'][i]
        t=myString.find('₹')
        if (t>=0):
            val=myString[t:t+7]
            fee_test1.at[i,'fee_new_info']=val
            
fee_test1['fee_new_info'].fillna(0, inplace=True)
fee_test1['fee_new_info']=fee_test1['fee_new_info'].str.replace('[^0-9]','')
fee_test1['fee_new_info']=fee_test1['fee_new_info'].str.replace('[^\w\s]','')
del fee_test1['Miscellaneous_Info']


result2=pd.concat([y_pred_test2, fee_test1], axis=1)
result2['fee_new_info'] = result2['fee_new_info'].str.strip()
result2['fee_new_info']=pd.to_numeric(result2.fee_new_info)
result2['fee_new_info'].fillna(0, inplace=True)

result2['fee_new_info'] = np.where(result2['fee_new_info']>=1000,100,result2['fee_new_info'])
result2['Fees']=np.where(result2['fee_new_info']==0,result2['Fee_pred'],result2['fee_new_info'])
del result2['Fee_pred']
del result2['fee_new_info']
result2.to_csv('submission_ohc_xgb_withadj.csv', index=False)

In [217]:
chk1=pd.read_excel('Final_Test.xlsx')
chk2=chk1['Qualification'].str.contains('inspired')
chk2=pd.DataFrame(chk2)
chk2['td']=np.where(chk2['Qualification']==True,100,0)
del chk2['Qualification']

result3=pd.concat([result2,chk2],axis=1)

result3['Fees']=result3['Fees']+result3['td']
del result3['td']

result3.to_excel('submission_ohc_xgb_withadj_v1.xlsx', index=False)#0.74267647

<h>**XGBoost_Term_Document_Matrix**</h>

In [9]:
df_new = pd.read_csv('df_tf_idf_wc.csv')

In [10]:
vec = CountVectorizer()
X = vec.fit_transform(df_new.Miscellaneous_Info10)
msc_df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
msc_df=pd.DataFrame(msc_df)

vec = CountVectorizer()
X = vec.fit_transform(df_new.Qualification10)
qual_df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
qual_df=pd.DataFrame(qual_df)

In [11]:
df_new_v1=pd.concat([qual_df,msc_df], axis=1)
df_new_v2=pd.concat([df_new,df_new_v1], axis=1)

del df_new_v2['Qualification10']
del df_new_v2['Miscellaneous_Info10']

In [12]:
prof1=pd.get_dummies(df_new_v2['Profile'],drop_first = True)
plac1=pd.get_dummies(df_new_v2['Places'],drop_first = True)
city1=pd.get_dummies(df_new_v2['City'],drop_first = True)

dfmerged=pd.concat([df_new_v2,prof1,plac1,city1],axis=1)

del dfmerged['Profile']
del dfmerged['Places']
del dfmerged['City']

dfmerged.columns = np.arange(len(dfmerged.columns))

train1=dfmerged[:5961]
test1=dfmerged[5961:7948]

X_train, X_validation, y_train, y_validation = train_test_split(train1, target, train_size=0.7, random_state=1234)

In [13]:
model_tdm=xgb.XGBRegressor(colsample_bylevel=0.8, 
                       colsample_bytree=0.8,
                       gamma=1, 
                       learning_rate=0.01, 
                       max_depth=20,
                       min_child_weight=2, 
                       n_estimators=300, 
                       objective='reg:linear', 
                       reg_alpha=1, 
                       reg_lambda=1,
                       scale_pos_weight=1, 
                       seed=99, 
                       subsample=0.8,
                       silent=True)
model_tdm.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.8,
       colsample_bytree=0.8, gamma=1, learning_rate=0.01, max_delta_step=0,
       max_depth=20, min_child_weight=2, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=1, reg_lambda=1, scale_pos_weight=1, seed=99, silent=True,
       subsample=0.8)

In [14]:
print(model_tdm.score(X_train, y_train))
print(model_tdm.score(X_validation,y_validation))

0.6814211911779566
0.25353176818447254


In [15]:
# save model to file
pickle.dump(model_tdm, open("basic_xgb_tdm.pickle.dat", "wb"))

In [16]:
#Model Loading
xgb_tdm_ld = pickle.load(open("basic_xgb_tdm.pickle.dat", "rb"))

In [17]:
ypred = xgb_tdm_ld.predict(X_train)
ypred_val = xgb_tdm_ld.predict(X_validation)


print(sqrt(mean_squared_error(y_train, ypred)))
print(sqrt(mean_squared_error(y_validation, ypred_val)))

107.45724582331675
165.98220092806733


<h>**Data For Stacking**</h>

In [18]:
xgb_tdm_ld = pickle.load(open("basic_xgb_tdm.pickle.dat", "rb"))


y_all=xgb_tdm_ld.predict(train1)
y_all=pd.DataFrame(y_all)

y_all.columns=['xgb_tdm']

y_all.to_csv('xgb_tdm_stck_train.csv', index=False)

In [19]:
#Without Adjustment
y_pred_test2=xgb_tdm_ld.predict(test1)
y_pred_test2=pd.DataFrame(y_pred_test2)
y_pred_test2.columns=['Fee_pred']

<h>**Validation Data For Stacking**</h>

In [20]:
xgb_tdm_ld = pickle.load(open("basic_xgb_tdm.pickle.dat", "rb"))

y_predicted_test=xgb_tdm_ld.predict(test1)
y_predicted_test=pd.DataFrame(y_predicted_test)
y_predicted_test.columns=['xgb_tdm']
y_predicted_test.to_csv('xgb_tdm_stck_val.csv', index=False)

In [21]:
#With Adjustment
fee_test = pd.read_excel('Final_Test.xlsx')
fee_test=fee_test['Miscellaneous_Info']
fee_test1=pd.DataFrame(fee_test)

for i in range(1987):
    if (pd.isnull(fee_test1['Miscellaneous_Info'][i])==False):
        myString=fee_test1['Miscellaneous_Info'][i]
        t=myString.find('₹')
        if (t>=0):
            val=myString[t:t+7]
            fee_test1.at[i,'fee_new_info']=val
            
fee_test1['fee_new_info'].fillna(0, inplace=True)
fee_test1['fee_new_info']=fee_test1['fee_new_info'].str.replace('[^0-9]','')
fee_test1['fee_new_info']=fee_test1['fee_new_info'].str.replace('[^\w\s]','')
del fee_test1['Miscellaneous_Info']


result2=pd.concat([y_pred_test2, fee_test1], axis=1)
result2['fee_new_info'] = result2['fee_new_info'].str.strip()
result2['fee_new_info']=pd.to_numeric(result2.fee_new_info)
result2['fee_new_info'].fillna(0, inplace=True)

result2['fee_new_info'] = np.where(result2['fee_new_info']>=1000,100,result2['fee_new_info'])
result2['Fees']=np.where(result2['fee_new_info']==0,result2['Fee_pred'],result2['fee_new_info'])
del result2['Fee_pred']
del result2['fee_new_info']
result2.to_csv('submission_tdm_xgb_withadj.csv', index=False)

In [22]:
chk1=pd.read_excel('Final_Test.xlsx')
chk2=chk1['Qualification'].str.contains('inspired')
chk2=pd.DataFrame(chk2)
chk2['td']=np.where(chk2['Qualification']==True,100,0)
del chk2['Qualification']

result3=pd.concat([result2,chk2],axis=1)

result3['Fees1']=np.where(result3['td']==100,100,result3['Fees'])
del result3['Fees']
del result3['td']

result3.columns=['Fees']

result3.to_excel('submission_tdm_xgb_withadj_v1.xlsx', index=False)#0.74462449

<h>**TF_IDF**</h>

In [232]:
df_new = pd.read_csv('df_tf_idf_wc.csv')

In [233]:
v = TfidfVectorizer()
x = v.fit_transform(df_new.Miscellaneous_Info10)
msc_df1 = pd.DataFrame(x.toarray(), columns=v.get_feature_names())
msc_df1=pd.DataFrame(msc_df1)

v = TfidfVectorizer()
x = v.fit_transform(df_new.Qualification10)
qual_df1 = pd.DataFrame(x.toarray(), columns=v.get_feature_names())
qual_df1=pd.DataFrame(qual_df1)

C:\Users\pf416e\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [234]:
df_new_v1=pd.concat([qual_df,msc_df], axis=1)
df_new_v2=pd.concat([df_new,df_new_v1], axis=1)

del df_new_v2['Qualification10']
del df_new_v2['Miscellaneous_Info10']

In [235]:
prof1=pd.get_dummies(df_new_v2['Profile'],drop_first = True)
plac1=pd.get_dummies(df_new_v2['Places'],drop_first = True)
city1=pd.get_dummies(df_new_v2['City'],drop_first = True)

dfmerged=pd.concat([df_new_v2,prof1,plac1,city1],axis=1)

del dfmerged['Profile']
del dfmerged['Places']
del dfmerged['City']

dfmerged.columns = np.arange(len(dfmerged.columns))

train1=dfmerged[:5961]
test1=dfmerged[5961:7948]

X_train, X_validation, y_train, y_validation = train_test_split(train1, target, train_size=0.7, random_state=1234)

In [236]:
model_tfidf=xgb.XGBRegressor(colsample_bylevel=0.8, 
                       colsample_bytree=0.8,
                       gamma=2, 
                       learning_rate=0.05, 
                       max_depth=12,
                       min_child_weight=4, 
                       n_estimators=500, 
                       objective='reg:linear', 
                       reg_alpha=1, 
                       reg_lambda=1,
                       scale_pos_weight=1, 
                       seed=99, 
                       subsample=0.8,
                       silent=True)
model_tfidf.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.8,
       colsample_bytree=0.8, gamma=2, learning_rate=0.05, max_delta_step=0,
       max_depth=12, min_child_weight=4, missing=None, n_estimators=500,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=1, reg_lambda=1, scale_pos_weight=1, seed=99, silent=True,
       subsample=0.8)

In [237]:
print(model_tfidf.score(X_train, y_train))
print(model_tfidf.score(X_validation,y_validation))

0.7220599878947132
0.24371649304521137


In [238]:
# save model to file
pickle.dump(model_tfidf, open("basic_xgb_tfidf.pickle.dat", "wb"))

In [239]:
#Model Loading
xgb_tfidf_ld = pickle.load(open("basic_xgb_tfidf.pickle.dat", "rb"))

In [240]:
ypred = xgb_tfidf_ld.predict(X_train)
ypred_val = xgb_tfidf_ld.predict(X_validation)


print(sqrt(mean_squared_error(y_train, ypred)))
print(sqrt(mean_squared_error(y_validation, ypred_val)))

100.36974001688573
167.06988315194656


<h>**Data For Stacking**</h>

In [241]:
xgb_tfidf_ld = pickle.load(open("basic_xgb_tfidf.pickle.dat", "rb"))

y_all=xgb_tfidf_ld.predict(train1)
y_all=pd.DataFrame(y_all)

y_all.columns=['xgb_tf']

y_all.to_csv('xgb_tf_stck_train.csv', index=False)

In [242]:
#Without Adjustment
y_pred_test2=xgb_tfidf_ld.predict(test1)
y_pred_test2=pd.DataFrame(y_pred_test2)
y_pred_test2.columns=['Fee_pred']

<h>**Validation Data For Stacking**</h>

In [243]:
xgb_tfidf_ld = pickle.load(open("basic_xgb_tfidf.pickle.dat", "rb"))

y_predicted_test=xgb_tfidf_ld.predict(test1)
y_predicted_test=pd.DataFrame(y_predicted_test)
y_predicted_test.columns=['xgb_tf']
y_predicted_test.to_csv('xgb_tf_stck_val.csv', index=False)

In [244]:
#With Adjustment
fee_test = pd.read_excel('Final_Test.xlsx')
fee_test=fee_test['Miscellaneous_Info']
fee_test1=pd.DataFrame(fee_test)

for i in range(1987):
    if (pd.isnull(fee_test1['Miscellaneous_Info'][i])==False):
        myString=fee_test1['Miscellaneous_Info'][i]
        t=myString.find('₹')
        if (t>=0):
            val=myString[t:t+7]
            fee_test1.at[i,'fee_new_info']=val
            
fee_test1['fee_new_info'].fillna(0, inplace=True)
fee_test1['fee_new_info']=fee_test1['fee_new_info'].str.replace('[^0-9]','')
fee_test1['fee_new_info']=fee_test1['fee_new_info'].str.replace('[^\w\s]','')
del fee_test1['Miscellaneous_Info']


result2=pd.concat([y_pred_test2, fee_test1], axis=1)
result2['fee_new_info'] = result2['fee_new_info'].str.strip()
result2['fee_new_info']=pd.to_numeric(result2.fee_new_info)
result2['fee_new_info'].fillna(0, inplace=True)

result2['fee_new_info'] = np.where(result2['fee_new_info']>=1000,100,result2['fee_new_info'])
result2['Fees']=np.where(result2['fee_new_info']==0,result2['Fee_pred'],result2['fee_new_info'])
del result2['Fee_pred']
del result2['fee_new_info']
result2.to_csv('submission_tfidf_xgb_withadj.csv', index=False)

In [245]:
chk1=pd.read_excel('Final_Test.xlsx')
chk2=chk1['Qualification'].str.contains('inspired')
chk2=pd.DataFrame(chk2)
chk2['td']=np.where(chk2['Qualification']==True,100,0)
del chk2['Qualification']

result3=pd.concat([result2,chk2],axis=1)

result3['Fees']=result3['Fees']+result3['td']
del result3['td']

result3.to_excel('submission_tfidf_xgb_withadj_v1.xlsx', index=False) #0.73412535